# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Load data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# Define features

In [3]:
features = ['RhythmScore', 'AudioLoudness', 'VocalContent', 'AcousticQuality', 
            'InstrumentalScore', 'LivePerformanceLikelihood', 'MoodScore', 
            'TrackDurationMs', 'Energy']
X = train[features]
y = train['BeatsPerMinute']
X_test = test[features]

# Feature engineering: Log-transform skewed features

In [4]:
X['AudioLoudness'] = np.log1p(-X['AudioLoudness'])
X_test['AudioLoudness'] = np.log1p(-X_test['AudioLoudness'])
X['TrackDurationMs'] = np.log1p(X['TrackDurationMs'])
X_test['TrackDurationMs'] = np.log1p(X_test['TrackDurationMs'])

C:\Users\nishi\AppData\Local\Temp\ipykernel_11896\3028610528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['AudioLoudness'] = np.log1p(-X['AudioLoudness'])
C:\Users\nishi\AppData\Local\Temp\ipykernel_11896\3028610528.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['AudioLoudness'] = np.log1p(-X_test['AudioLoudness'])
C:\Users\nishi\AppData\Local\Temp\ipykernel_11896\3028610528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

# Split data (80/20 for holdout validation)

In [5]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)
X_test_scaled = scaler.transform(X_test)

# Train Individual Models

# Linear Regression

In [7]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr_holdout_pred = lr.predict(X_holdout_scaled)
lr_rmse = np.sqrt(mean_squared_error(y_holdout, lr_holdout_pred))
print(f"Linear RMSE: {lr_rmse:.4f}")

Linear RMSE: 26.4440


# Random Forest (your best params)

In [8]:
rf = RandomForestRegressor(n_estimators=150, max_depth=10, min_samples_split=5, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)
rf_holdout_pred = rf.predict(X_holdout_scaled)
rf_rmse = np.sqrt(mean_squared_error(y_holdout, rf_holdout_pred))
print(f"RF RMSE: {rf_rmse:.4f}")

RF RMSE: 26.4398


# XGBoost with adjusted params

In [9]:
xgb_model = xgb.XGBRegressor(n_estimators=140, learning_rate=0.07, max_depth=6, random_state=42)
xgb_model.fit(X_train_scaled, y_train)
xgb_holdout_pred = xgb_model.predict(X_holdout_scaled)
xgb_rmse = np.sqrt(mean_squared_error(y_holdout, xgb_holdout_pred))
print(f"XGBoost RMSE: {xgb_rmse:.4f}")

XGBoost RMSE: 26.4566


# Cross-Validation for Weight Optimization (5-fold)

In [10]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_rmse = float('inf')
best_weights = None
for train_idx, val_idx in kf.split(X_train_scaled):
    X_train_fold, X_val_fold = X_train_scaled[train_idx], X_train_scaled[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    lr_fold_pred = lr.predict(X_val_fold)
    rf_fold_pred = rf.predict(X_val_fold)
    xgb_fold_pred = xgb_model.predict(X_val_fold)
    
    weight_combos = [
        (0.0975, 0.6025, 0.3000), (0.0980, 0.6020, 0.3000), (0.0985, 0.6015, 0.3000),
        (0.0975, 0.6020, 0.3005), (0.0980, 0.6015, 0.3005), (0.0985, 0.6010, 0.3005)
    ]  # (LR, RF, XGB) summing to 1
    for w_lr, w_rf, w_xgb in weight_combos:
        ensemble_fold_pred = w_lr * lr_fold_pred + w_rf * rf_fold_pred + w_xgb * xgb_fold_pred
        fold_rmse = np.sqrt(mean_squared_error(y_val_fold, ensemble_fold_pred))
        if fold_rmse < best_rmse:
            best_rmse = fold_rmse
            best_weights = (w_lr, w_rf, w_xgb)

print(f"Best Weights (5-fold CV-averaged): {best_weights}, Best CV RMSE: {best_rmse:.4f}")

Best Weights (5-fold CV-averaged): (0.0975, 0.602, 0.3005), Best CV RMSE: 26.2483


# Generate test predictions with best weights

In [11]:
lr_test_pred = lr.predict(X_test_scaled)
rf_test_pred = rf.predict(X_test_scaled)
xgb_test_pred = xgb_model.predict(X_test_scaled)
test_pred = (best_weights[0] * lr_test_pred + best_weights[1] * rf_test_pred + 
             best_weights[2] * xgb_test_pred)
submission['BeatsPerMinute'] = test_pred
submission.to_csv('submission_ensemble_refined_tuned.csv', index=False)
print("Submission file created: submission_ensemble_refined_tuned.csv")

Submission file created: submission_ensemble_refined_tuned.csv
